In [7]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
import random
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from pywt import wavedec

In [8]:
import os
dataset_path = '../Preprocessing/baru/'
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    for d in dirnames:
      for j, (dirpath2, dirnames2, filenames2) in enumerate(os.walk(os.path.join(dirpath, d))):
        for f in filenames2:
          print(os.path.join(dirpath2, f))

../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L5.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L7.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L8.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L1.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L2.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L9.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L4.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L10.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L3.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L6.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L4.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L6.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L5.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L8.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L1.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L9.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L7.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L3.csv
../Preprocessing/baru/db5/d

In [9]:
import os
dataset_path = '../Preprocessing/baru'
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    for d in dirnames:
      for j, (dirpath2, dirnames2, filenames2) in enumerate(os.walk(os.path.join(dirpath, d))):
        for f in filenames2:
          file_path = os.path.join(dirpath2, f)
          print("Processing file: ", file_path)
          df = pd.read_csv(file_path)

          X = df.drop(['label', 'filename'], axis=1)
          y = df.iloc[:, -1]

          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

          lb = LabelEncoder()
          X_train = np.expand_dims(X_train, axis=2)
          X_test = np.expand_dims(X_test, axis=2)
          y_test = tf.keras.utils.to_categorical(lb.fit_transform(y_test))
          y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))

          model = tf.keras.Sequential([
              tf.keras.layers.LSTM(64, input_shape=(X_train.shape[1],X_train.shape[2]), return_sequences=True),
              tf.keras.layers.Dense(32,activation='relu', activity_regularizer=tf.keras.regularizers.l2(1e-4)),

              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(64, activation='relu', activity_regularizer=tf.keras.regularizers.l2(1e-4)),
              tf.keras.layers.Dense(32, activation='relu', activity_regularizer=tf.keras.regularizers.l2(1e-4)),
              
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.Dense(5, activation='softmax')
          ])

          optimazer = tf.keras.optimizers.Adam(learning_rate=0.0001)
          model.compile(optimizer=optimazer, loss='categorical_crossentropy', metrics=['acc'])

          history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

          test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

          to_append = f'{file_path}'
          to_append += f' {test_accuracy}'
          file = open('../Preprocessing/hasil.csv', 'a', newline='')
          with file:
              writer = csv.writer(file)
              writer.writerow(to_append.split())

Processing file:  ../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L5.csv
Epoch 1/50
20/20 [==============================] - 3s 47ms/step - loss: 1.2442 - acc: 0.5172 - val_loss: 1.5393 - val_acc: 0.3438
Epoch 2/50
20/20 [==============================] - 1s 25ms/step - loss: 0.7622 - acc: 0.7719 - val_loss: 1.4994 - val_acc: 0.5063
Epoch 3/50
20/20 [==============================] - 0s 24ms/step - loss: 0.6200 - acc: 0.8313 - val_loss: 1.4702 - val_acc: 0.6500
Epoch 4/50
20/20 [==============================] - 0s 24ms/step - loss: 0.5310 - acc: 0.8641 - val_loss: 1.4388 - val_acc: 0.7188
Epoch 5/50
20/20 [==============================] - 0s 24ms/step - loss: 0.4751 - acc: 0.8922 - val_loss: 1.4067 - val_acc: 0.8000
Epoch 6/50
20/20 [==============================] - 0s 25ms/step - loss: 0.4266 - acc: 0.9031 - val_loss: 1.3702 - val_acc: 0.8500
Epoch 7/50
20/20 [==============================] - 0s 24ms/step - loss: 0.3789 - acc: 0.9187 - val_loss: 1.3348 - val_acc: 0.8562
Epoch 8/50
